In [1]:
from utils import Load

path = "./data/e-comerce_Olist_dataset"
load = Load(path)

# Cargamos todos los csv a un diccionario que alojara los df
data_dict = load.load_from_csv(path)

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import os  


In [3]:
# traemos aqui los csv
clientes = data_dict['olist_customers_dataset']
geolocalizacion = data_dict['olist_geolocation_dataset']
items = data_dict['olist_order_items_dataset']
metodo_de_pago = data_dict['olist_order_payments_dataset']
reviews = data_dict['olist_order_reviews_dataset']
ordenes =data_dict['olist_orders_dataset']
productos = data_dict['olist_products_dataset']
vendedores = data_dict['olist_sellers_dataset']
productos_info = data_dict['product_category_name_translation']

In [4]:
# Creamos las carpetas para guardar nuestros archivos etl y auxiliar que usaremos mas adelante
path_etl = "Dataset_etl"
path_aux = "Dataset_aux"
os.makedirs(path_etl, exist_ok=True)  
os.makedirs(path_aux, exist_ok=True)  

## ETL Gelolocalizacion

In [12]:
def etl_geolocalizacion(geolocalizacion):
    """
    Esta funcion le realiza ETL a la tabla geolocalizacion, normalizando latitudes y longitudes tomando como parametro
    que la mayor parte de brasil se halla dentro del hemisferio sur, entre los paralelos 5.5 de latitud N, y los -34 de latitud S; y entre los meridianos que señalan los -32 y los -74 de longitud.
    En un segundo paso crea un nuevo dataframe con zip codes unicos y se les asigna un ID.
    retorna el dataframe de geolocalizacion al que solo se le realiza el ETL, y el df filtrado para ser utilizados luego en el proceso de etl de vendedores y clientes
    
    """
    # Seleccionamos los outliers. 
    out_latylon = geolocalizacion [(geolocalizacion.geolocation_lat > 5.5)| (geolocalizacion.geolocation_lat < -34)|(geolocalizacion.geolocation_lng > -32 )| (geolocalizacion.geolocation_lng  < -74)]

    # generamos un user random
    import string, random
    user = ''.join(random.choice(string.ascii_lowercase) for i in range(6))

    #Usamos Geopy
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent= user)
    location_city = out_latylon.geolocation_city.values
    index = out_latylon.index
    for i in range(0,len(location_city)):
        location = geolocator.geocode(location_city[i]+', Brasil')
        try:
            geolocalizacion.loc[geolocalizacion.index==index[i], 'geolocation_lng'] = location.longitude
            geolocalizacion.loc[geolocalizacion.index==index[i], 'geolocation_lat'] = location.latitude
        except:
            try:
                location = geolocator.geocode(location_city[i])
                geolocalizacion.loc[geolocalizacion.index==index[i], 'geolocation_lng'] = location.longitude
                geolocalizacion.loc[geolocalizacion.index==index[i], 'geolocation_lat'] = location.latitude
            except:
                geolocalizacion.loc[geolocalizacion.index==index[i], 'geolocation_lat'] = geolocalizacion[geolocalizacion.geolocation_city==location_city[i]].geolocation_lat.mean()
                geolocalizacion.loc[geolocalizacion.index==index[i], 'geolocation_lng'] = geolocalizacion[geolocalizacion.geolocation_city==location_city[i]].geolocation_lng.mean()

    # ahora no es realmente necesario exportar el corregido.. pero si quisieramos podemos ejecutar el codigo:
    geolocalizacion.to_csv('data\e-comerce_Olist_dataset\olist_geolocation_dataset_coregido.csv', index_label=False)

    # Agrupamos por geolocation_zip_code_prefix
    geoloc_filtrado = geolocalizacion.groupby('geolocation_zip_code_prefix').mean().reset_index()
    # agregamos columna de Id igual a la de index
    geoloc_filtrado['IdGeolocalizacion'] = geoloc_filtrado.index

    # Agregamos Ciudad y Estado
    geoloc_filtrado['Ciudad'] = geoloc_filtrado.apply(lambda r: geolocalizacion[geolocalizacion.geolocation_zip_code_prefix==r.geolocation_zip_code_prefix].geolocation_city.values[0], axis =1)
    geoloc_filtrado['Estado'] = geoloc_filtrado.apply(lambda r: geolocalizacion[geolocalizacion.geolocation_zip_code_prefix==r.geolocation_zip_code_prefix].geolocation_state.values[0], axis =1)

    # renombramos y reordenamos las columnas
    geoloc_filtrado.rename(columns={'geolocation_zip_code_prefix': 'zip_code_prefix', 
                                    'geolocation_lat': 'Latitud',
                                    'geolocation_lng': 'Longitud',
                                    'IdGeolocalizacion':'Id_Geolocalizacion'}, inplace=True)
    geoloc_filtrado = geoloc_filtrado.reindex(columns=['Id_Geolocalizacion', 'zip_code_prefix', 'Latitud', 
                                    'Longitud','Ciudad','Estado'])


    # En la carpeta Dataset_etl guardamos el archivos de geolocalizacion luego del proceso de ETL
    geoloc_filtrado.to_csv("{}/geolocalizacion_etl.csv".format(path_etl), index=False)

    #Retornamos el df de geolocalizacion, de la primer parte del etl, porque es el que necesitamos para el etl de clientes y vendedores
    return geolocalizacion, geoloc_filtrado

In [13]:
geolocalizacion,  geoloc_filtrado = etl_geolocalizacion(geolocalizacion)

### ETL a clientes

In [16]:
def clientes_etl(clientes):
    """
    Funcion que realiza el ETL a la tabla clientes. 
        Se normalizan los codigos postales que no coinciden con los de la tabla geolocalizacion
        Para los zip codes de clientes que no se encuentran en geolocalizacion pero si se reconoce su ciudad
        se reemplazara el zip_code por la moda del zip_code correspondiente a dicha ciudad en geolocalizacion.
        Como resultado guardara dos archivos 
            clientes_etl.csv con el resultado del etl 
            clientes_aux.csv con los datos que no logro resolver
    """
    # usamos el df de geolocalizacion porque no tiene filtrados los zip_codes y por ende hay mas datos para calcular el modo
    geoloc_unique_code = geolocalizacion.geolocation_zip_code_prefix.unique()
    clientes_unique_code = clientes.customer_zip_code_prefix.unique()
    zip_codes_missing = []
    for item in clientes_unique_code:
        if item not in geoloc_unique_code:
            try:
                city = clientes.loc[clientes['customer_zip_code_prefix'] == item].customer_city.values[0]
                zip_code = geolocalizacion[geolocalizacion.geolocation_city==city].geolocation_zip_code_prefix.mode()[0]
                clientes['customer_zip_code_prefix'] = clientes['customer_zip_code_prefix'].replace(item, zip_code)
            except:
                zip_codes_missing.append(item)
    
    # buscamos los index de los codigos que no logramos resolver para luego guardarlos en una tabla auxiliar
    indexes = []
    for item in zip_codes_missing:
        ind = clientes[clientes.customer_zip_code_prefix==item].index.values
        for i in ind:
            indexes.append(i)

    cientes_aux = clientes.iloc[indexes]
    clientes.drop(index = indexes, axis=0, inplace=True)

    # Ahora que eliminamos estos valores problema aplicamos el id_Geolocalizacion a la tabla clientes
    clientes['Id_Geolocalizacion'] = clientes.apply(lambda r: geoloc_filtrado[geoloc_filtrado.zip_code_prefix==r.customer_zip_code_prefix].Id_Geolocalizacion.values[0], axis = 1)
    clientes = clientes.drop(['customer_zip_code_prefix','customer_city','customer_state'],axis=1)

    # guardamos clientes_aux en un csv
    cientes_aux.to_csv("{}/cientes_aux.csv".format(path_aux), index=False)

    # Guardamos las etl
    clientes.to_csv("{}/clientes_etl.csv".format(path_etl), index=False)

In [17]:
clientes_etl(clientes)

### ETL Vendedores

In [18]:
def vendedores_etl(vendedores):
    """
    Funcion que realiza el ETL a la tabla vendedores. 
        Se normalizan los codigos postales que no coinciden con los de la tabla geolocalizacion
        Para los zip codes de vendedores que no se encuentran en geolocalizacion pero si se reconoce su ciudad
        se reemplazara el zip_code por la moda del zip_code correspondiente a dicha ciudad en geolocalizacion.
        Como resultado guardara un archivo vendedores_etl.csv con el resultado del etl 
    """

    # Hay 7 codigos postales de vendedores que no coinciden con los codigos de la tabla geolocalizacion.
    # Para cada ciudad en la tabla de geolocalizacion hay mas de un zip-code... 
    # lo que vamos a hacer es cambiar los de los vendedores por el modo de los de geolocalizacion  
    # usamos el df de geolocalizacion porque no tiene filtrados los zip_codes y por ende hay mas datos para calcular el modo
 
    geoloc_unique_code = geolocalizacion.geolocation_zip_code_prefix.unique()
    vendedores_unique_code = vendedores.seller_zip_code_prefix.unique()
    for item in vendedores_unique_code:
        if item not in geoloc_unique_code:
            city = vendedores.loc[vendedores['seller_zip_code_prefix'] == item].seller_city.values[0]
            zip_code = geolocalizacion[geolocalizacion.geolocation_city==city].geolocation_zip_code_prefix.mode()[0]
            vendedores['seller_zip_code_prefix'] = vendedores['seller_zip_code_prefix'].replace(item, zip_code)
    
    #Logramos arreglar todos :) Ahora aplicamos el id_Geolocalizacion a la tabla vendedores

    vendedores['Id_Geolocalizacion'] = vendedores.apply(lambda r: geoloc_filtrado[geoloc_filtrado.zip_code_prefix==r.seller_zip_code_prefix].Id_Geolocalizacion.values[0], axis = 1)
    vendedores = vendedores.drop(['seller_zip_code_prefix','seller_city','seller_state'],axis=1)

    
    # Guardamos las etl
    vendedores.to_csv("{}/vendedores_etl.csv".format(path_etl), index=False)


In [19]:
vendedores_etl(vendedores)

## Metodo de pago // tipo de pago

In [6]:
def metodo_tipo_pago(metodo_de_pago):
    """
    Se agregan los payment_type_id y se crean las tablas normalizadas payment_type y metodo_de_pago_corregido a partir de la tabla metodo_de_pago
    """
    payment_type = metodo_de_pago['payment_type']
    tipos_pago = {j:i+1 for i,j in enumerate(payment_type.unique())}
    payment_type_df = pd.DataFrame(
        {
            'payment_type_id': tipos_pago.values(),
            'payment_type': tipos_pago.keys()
        })
    metodo_de_pago['payment_type'] = metodo_de_pago['payment_type'].apply(lambda x: tipos_pago[x])
    metodo_de_pago.rename(columns={'payment_type': 'payment_type_id'}, inplace=True)

    #exportamos a csv 
    payment_type_df.to_csv('{}/payment_type.csv'.format(path_etl), index=False)
    metodo_de_pago.to_csv('{}/metodo_de_pago_corregido.csv'.format(path_etl), index=False)

In [7]:
metodo_tipo_pago(metodo_de_pago)

## Ordenes // estado de la orden

In [5]:
def ordenes_estado_orden (ordenes):
    """
    Se agregan los order_status_id y se crean las tablas normalizadas order_status y ordenes_corregido a partir de la tabla ordenes
    """
    estado_orden = ordenes['order_status'].unique()
    estado_orden = {j:i+1 for i,j in enumerate(estado_orden)}
    order_status_df = pd.DataFrame(
        {
            'order_status_id': estado_orden.values(),
            'order_status': estado_orden.keys()
        })
    ordenes['order_status'] = ordenes['order_status'].apply(lambda x: estado_orden[x])
    ordenes.rename(columns={'order_status': 'order_status_id'}, inplace=True)

    # Exportamos los csv
    ordenes.to_csv('./Dataset_etl/ordenes_corregido.csv', index=False)
    order_status_df.to_csv('./Dataset_etl/order_status.csv', index=False)
    return ordenes


In [6]:
ordenes = ordenes_estado_orden(ordenes)

## Productos // tipo de producto

In [10]:
def productos_tipo_producto(productos):
    """
    Se agregan product_category_name_id y se crean las tablas normalizadas product_category_name y productos_corregido a partir de la tabla productos
    """

    productos_name = productos['product_category_name'].unique()
    productos_name = {j:i+1 for i,j in enumerate(productos_name)}

    product_category_name_df = pd.DataFrame(
        {
            'product_category_name_id': productos_name.values(),
            'category_name': productos_name.keys()
        })
    productos['product_category_name'] = productos['product_category_name'].apply(lambda x: productos_name[x])
    productos.rename(columns={'product_category_name': 'product_category_name_id'}, inplace=True)
    
    # Exportamos a csv
    productos.to_csv('./Dataset_etl/productos_corregido.csv', index=False)
    product_category_name_df.to_csv('./Dataset_etl/product_category_name.csv', index=False)



In [11]:
productos_tipo_producto(productos)

### ETL Ordenes
#### Trabajo con ordenes luego de agregar order_status_id
 - ordenes = ordenes_estado_orden(ordenes)

In [7]:
ordenes.head(3)

,order_id,customer_id,order_status_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,1,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,1,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,1,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04


In [8]:
ordenes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status_id                99441 non-null  int64         
 3   order_purchase_timestamp       99441 non-null  object        
 4   order_approved_at              99281 non-null  object        
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(4)
memory usage: 6.1+ MB


al importar los datos no toma `order_purchase_timestamp` y `order_approved_at` como tiempo. los cambio manualmente aqui


In [9]:
ordenes['order_purchase_timestamp'] = pd.to_datetime(ordenes['order_purchase_timestamp'])
ordenes['order_approved_at'] = pd.to_datetime(ordenes['order_approved_at'])

In [10]:
ordenes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status_id                99441 non-null  int64         
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), int64(1), object(2)
memory usage: 6.1+ MB


In [17]:
ordenes.shape

(99441, 8)

In [11]:
ordenes.isnull().sum()

order_id                            0
customer_id                         0
order_status_id                     0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Tenemos 99441 filas y Encontramos valores nulos en 
- `order_approved_at` 160 nulos
- `order_delivered_carrier_date` 1783 nulos
- `order_delivered_customer_date` 2965 nulos

In [12]:
import datetime
date= datetime.timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0) 
date

datetime.timedelta(0)

Las diferencias a evaluar me indican: 
- **Hora de pago aprobado y compra**  < 0 : indica que el pago fue aprobado antes de realizar la compra
- **despacho y compra** < 0 : indica que el  pedido fue despachado antes de realizar la compra
- **entrega y compra**  < 0 : indica que el  pedido fue entregado antes de realizar la compra


In [13]:
ordenes_outliers = ordenes.copy()
ordenes_outliers["diff_pagoaprobado_compra"] = ordenes_outliers.order_approved_at - ordenes_outliers.order_purchase_timestamp
ordenes_outliers["diff_despacho_compra"] = ordenes_outliers.order_delivered_carrier_date - ordenes_outliers.order_purchase_timestamp
ordenes_outliers["diff_entregareal_compra"] = ordenes_outliers.order_delivered_customer_date - ordenes_outliers.order_purchase_timestamp


In [14]:
ordenes_outliers_select =  ordenes_outliers[(ordenes_outliers.diff_pagoaprobado_compra < date) | (ordenes_outliers.diff_despacho_compra <date )| (ordenes_outliers.diff_entregareal_compra < date)]

Exploro los outliers buscando el porque de las diferencias

In [15]:
ordenes_outliers_select.head()

,order_id,customer_id,order_status_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,diff_pagoaprobado_compra,diff_despacho_compra,diff_entregareal_compra
615,b9afddbdcfadc9a87b41a83271c3e888,85c6af75161b8b2b1af98e82b5a6a5a5,1,2018-08-16 13:50:48,2018-08-16 14:05:13,2018-08-16 13:27:00,2018-08-24 14:58:37,2018-09-04,0 days 00:14:25,-1 days +23:36:12,8 days 01:07:49
1111,ad133696906f6a78826daa0911b7daec,e6f5b234bb0d847f10eebd70130c5d49,1,2018-06-15 15:41:22,2018-06-15 16:19:23,2018-06-15 14:52:00,2018-06-22 18:09:37,2018-07-18,0 days 00:38:01,-1 days +23:10:38,7 days 02:28:15
1329,74e033208dc13a7b8127eb8e73d09b76,72fcbb1145f2889eddcba6d6c1d6c090,1,2018-05-02 10:48:44,2018-05-02 11:13:45,2018-05-02 09:49:00,2018-05-07 23:06:36,2018-05-29,0 days 00:25:01,-1 days +23:00:16,5 days 12:17:52
1372,a6b58794fd2ba533359a76c08df576e3,ccd8e3459ad58ae538c9d2cf35532ba4,1,2018-05-14 15:18:23,2018-05-14 15:33:35,2018-05-14 13:46:00,2018-05-19 19:33:32,2018-06-08,0 days 00:15:12,-1 days +22:27:37,5 days 04:15:09
1864,5792e0b1c8c8a2bf53af468c9a422c88,5c427f06e7fcf902e309743516e2c580,1,2018-07-26 13:25:14,2018-07-26 13:35:14,2018-07-26 12:42:00,2018-07-30 14:45:02,2018-08-09,0 days 00:10:00,-1 days +23:16:46,4 days 01:19:48


In [16]:
ordenes_outliers_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 615 to 99230
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   order_id                       166 non-null    object         
 1   customer_id                    166 non-null    object         
 2   order_status_id                166 non-null    int64          
 3   order_purchase_timestamp       166 non-null    datetime64[ns] 
 4   order_approved_at              166 non-null    datetime64[ns] 
 5   order_delivered_carrier_date   166 non-null    datetime64[ns] 
 6   order_delivered_customer_date  165 non-null    datetime64[ns] 
 7   order_estimated_delivery_date  166 non-null    datetime64[ns] 
 8   diff_pagoaprobado_compra       166 non-null    timedelta64[ns]
 9   diff_despacho_compra           166 non-null    timedelta64[ns]
 10  diff_entregareal_compra        165 non-null    timedelta64[ns]
dtypes:

In [17]:
ordenes_outliers_select[ordenes_outliers_select.diff_entregareal_compra < date]

,order_id,customer_id,order_status_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,diff_pagoaprobado_compra,diff_despacho_compra,diff_entregareal_compra


In [18]:
ordenes_outliers_select[ordenes_outliers_select.diff_pagoaprobado_compra < date]

,order_id,customer_id,order_status_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,diff_pagoaprobado_compra,diff_despacho_compra,diff_entregareal_compra


In [19]:
ordenes_outliers_select[ordenes_outliers_select.diff_despacho_compra < date]

,order_id,customer_id,order_status_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,diff_pagoaprobado_compra,diff_despacho_compra,diff_entregareal_compra
615,b9afddbdcfadc9a87b41a83271c3e888,85c6af75161b8b2b1af98e82b5a6a5a5,1,2018-08-16 13:50:48,2018-08-16 14:05:13,2018-08-16 13:27:00,2018-08-24 14:58:37,2018-09-04,0 days 00:14:25,-1 days +23:36:12,8 days 01:07:49
1111,ad133696906f6a78826daa0911b7daec,e6f5b234bb0d847f10eebd70130c5d49,1,2018-06-15 15:41:22,2018-06-15 16:19:23,2018-06-15 14:52:00,2018-06-22 18:09:37,2018-07-18,0 days 00:38:01,-1 days +23:10:38,7 days 02:28:15
1329,74e033208dc13a7b8127eb8e73d09b76,72fcbb1145f2889eddcba6d6c1d6c090,1,2018-05-02 10:48:44,2018-05-02 11:13:45,2018-05-02 09:49:00,2018-05-07 23:06:36,2018-05-29,0 days 00:25:01,-1 days +23:00:16,5 days 12:17:52
1372,a6b58794fd2ba533359a76c08df576e3,ccd8e3459ad58ae538c9d2cf35532ba4,1,2018-05-14 15:18:23,2018-05-14 15:33:35,2018-05-14 13:46:00,2018-05-19 19:33:32,2018-06-08,0 days 00:15:12,-1 days +22:27:37,5 days 04:15:09
1864,5792e0b1c8c8a2bf53af468c9a422c88,5c427f06e7fcf902e309743516e2c580,1,2018-07-26 13:25:14,2018-07-26 13:35:14,2018-07-26 12:42:00,2018-07-30 14:45:02,2018-08-09,0 days 00:10:00,-1 days +23:16:46,4 days 01:19:48
...,...,...,...,...,...,...,...,...,...,...,...
98172,f7780ea2807db31691e83f0013294035,3088cc4bfafe311f1d84830808c2aa6d,1,2018-07-30 15:22:15,2018-07-30 15:35:16,2018-07-30 15:00:00,2018-08-02 18:32:30,2018-08-02,0 days 00:13:01,-1 days +23:37:45,3 days 03:10:15
98430,d7646ffe8fdd9e7d9557f9f7cbf04530,f896e70d84f6db9a56b86a47dd8718f0,1,2018-05-04 14:50:37,2018-05-04 15:10:22,2018-05-04 14:48:00,2018-05-08 19:06:42,2018-05-16,0 days 00:19:45,-1 days +23:57:23,4 days 04:16:05
98672,5ded8a3706eabd813685534724f066de,460316e38d4d241fc79d4f6729f8f7f6,1,2018-07-18 08:46:52,2018-07-18 09:01:48,2018-07-18 08:44:00,2018-07-25 13:53:17,2018-08-10,0 days 00:14:56,-1 days +23:57:08,7 days 05:06:25
98780,d10046876c7d9f01613da59ffa6cb07f,6559cb5b365f173915f766fefcd86972,1,2018-07-18 16:14:16,2018-07-18 16:25:17,2018-07-18 15:34:00,2018-07-23 20:46:44,2018-08-07,0 days 00:11:01,-1 days +23:19:44,5 days 04:32:28


busco en google si hay diferencia horaria entre la ciudad del cliente y el vendedor. (no encuentro valores que indiquen esto)

In [20]:
clientes[clientes.customer_id == '3088cc4bfafe311f1d84830808c2aa6d']

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
1488,3088cc4bfafe311f1d84830808c2aa6d,2f0def88a867b9c0aabb60c561b4d640,9991,diadema,SP


In [21]:
items[items.order_id == 'f7780ea2807db31691e83f0013294035']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
108887,f7780ea2807db31691e83f0013294035,1,904a9e449fbfd90da738ab70ae6ca003,f262cbc1c910c83959f849465454ddd3,2018-08-01 15:35:16,59.99,7.68


In [22]:
vendedores[vendedores.seller_id == 'f262cbc1c910c83959f849465454ddd3' ]

,seller_id,seller_zip_code_prefix,seller_city,seller_state
1334,f262cbc1c910c83959f849465454ddd3,3564,sao paulo,SP


Separo en aux y etl la tabla clientes para evaluar despues que hacer...

In [25]:
indexes = ordenes_outliers_select.index.values

In [26]:
ordenes_aux = ordenes.iloc[indexes]
ordenes_etl = ordenes.drop(index = indexes, axis=0)

In [29]:
ordenes_etl.shape[0] + ordenes_aux.shape[0] - ordenes.shape[0]

0

In [30]:
# guardamos ordenes_aux en un csv
ordenes_aux.to_csv("{}/ordenes_aux.csv".format(path_aux), index=False)

# Guardamos ordenes_etl en un csv
ordenes_etl.to_csv("{}/ordenes_etl.csv".format(path_etl), index=False)